# Fato Estabelecimento

In [0]:
import pyspark.sql.functions as f

In [0]:
spark.sql('USE CATALOG saude_sus')

In [0]:
df_trusted = spark.read.table('saude_sus.trusted.tru_estabelecimento')
display(df_trusted)

In [0]:
print(df_trusted.columns)

In [0]:
df_fact = df_trusted.select(
    f.md5(f.concat_ws("-", f.col("COD_CNES"), f.col("DAT_ATUALIZACAO"))).alias("SK_ESTABELECIMENTO"),
    f.md5(f.col("COD_MUNICIPIO").cast("string")).alias("SK_LOCALIZACAO"),
    f.regexp_replace(f.col("DAT_ATUALIZACAO"), "-", "").cast("int").alias("SK_CALENDARIO"),
    f.md5(f.col("COD_IDENTIFICADOR_TURNO_ATENDIMENTO").cast("string")).alias("SK_TURNO"),
    f.md5(f.concat_ws("-", 
        f.col("COD_TIPO_UNIDADE"), 
        f.col("COD_ATIVIDADE_ENSINO")
    )).alias("SK_PERFIL_UNIDADE"),

    "COD_CNES",
    f.coalesce(f.col("FLG_POSSUI_ATENDIMENTO_AMBULATORIAL_SUS").cast("int"), f.lit(0)).alias("QTD_ATENDIMENTO_SUS"),
    f.coalesce(f.col("FLG_POSSUI_CENTRO_CIRURGICO").cast("int"), f.lit(0)).alias("QTD_CENTRO_CIRURGICO"),
    f.coalesce(f.col("FLG_POSSUI_CENTRO_OBSTETRICO").cast("int"), f.lit(0)).alias("QTD_CENTRO_OBSTETRICO"),
    f.coalesce(f.col("FLG_POSSUI_CENTRO_NEONATAL").cast("int"), f.lit(0)).alias("QTD_CENTRO_NEONATAL"),
    f.coalesce(f.col("FLG_POSSUI_ATENDIMENTO_HOSPITALAR").cast("int"), f.lit(0)).alias("QTD_ATENDIMENTO_HOSPITALAR"),
    f.coalesce(f.col("FLG_POSSUI_SERVICO_APOIO").cast("int"), f.lit(0)).alias("QTD_SERVICO_APOIO"),
    f.coalesce(f.col("FLG_POSSUI_ATENDIMENTO_AMBULATORIAL").cast("int"), f.lit(0)).alias("QTD_ATENDIMENTO_AMBULATORIAL")
)


display(df_fact)

In [0]:
from delta.tables import DeltaTable

if not spark.catalog.tableExists("saude_sus.refined.FACT_ESTABELECIMENTO"):
    df_fact.write.format("delta").saveAsTable("saude_sus.refined.FACT_ESTABELECIMENTO")
else:
    target_table = DeltaTable.forName(spark, "saude_sus.refined.FACT_ESTABELECIMENTO")
    
    target_table.alias("target").merge(
        df_fact.alias("source"),
        "target.COD_CNES = source.COD_CNES"
    ).whenMatchedUpdateAll(condition = "source.DAT_ATUALIZACAO > target.DAT_ATUALIZACAO") \
     .whenNotMatchedInsertAll() \
     .execute()